In [1]:
import pandas as pd
import numpy as np
import datetime
from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Bidirectional, LSTM, Dense, RepeatVector, TimeDistributed

# Read in Sentinel A, B data; Read in in-situ data

In [2]:
# Read in Sentinel A data
sentinel_data_A = pd.read_csv("data/Sentinel_3A_water_level_Version0.csv")
sentinel_data_A = sentinel_data_A.rename(
    columns={
        "Date (YYYYMMDD)" : "date",
        "Lake_name" : "lake_name",
        "Latitude" : "latitude",
        "Longitude" : "longitude",
        "Relaive_orbit" : "relative_orbit",
        "Lake water level (m)" : "lake_water_level"
    }
)
# Convert date to date time.
sentinel_data_A.loc[:, "date"] = pd.to_datetime(sentinel_data_A.loc[:, "date"], format="%Y%m%d")

In [3]:
# Read in Sentinel B data
sentinel_data_B = pd.read_csv("data/Sentinel_3B_water_level_Version0.csv")

sentinel_data_B = sentinel_data_B.rename(
    columns={
        "Date (YYYYMMDD)" : "date",
        "Lake_name" : "lake_name",
        "Latitude" : "latitude",
        "Longitude" : "longitude",
        "Relaive_orbit" : "relative_orbit",
        "Lake water level (m)" : "lake_water_level"
    }
)

# Convert date to date time.
sentinel_data_B.loc[:, "date"] = pd.to_datetime(sentinel_data_B.loc[:, "date"], format="%Y%m%d")

In [4]:
# Concatenate
sentinel_data = pd.concat([sentinel_data_A, sentinel_data_B])

In [5]:
# Isolate Lake Winnipeg data
lake_winnipeg = sentinel_data[
    sentinel_data["lake_name"] == "Winnipeg"
]

In [6]:
# Read in-situ data
lake_winnipeg_in_situ = pd.read_csv("./data/WinnipegLake_at_GeorgeIsland.csv")
lake_winnipeg_in_situ = lake_winnipeg_in_situ.rename(
    columns={
        "Date" : "date",
        "Value (m)" : "in_situ_lake_water_level"
    }
)

In [7]:
# Convert date to date time.
lake_winnipeg_in_situ.loc[:, "date"] = pd.to_datetime(lake_winnipeg_in_situ.loc[:, "date"], format="%Y-%m-%d")

# Select only date and in_situ_lake_water_level
lake_winnipeg_in_situ = lake_winnipeg_in_situ[["date", "in_situ_lake_water_level"]]

In [8]:
# Join the data on date
lake_winnipeg = lake_winnipeg.merge(lake_winnipeg_in_situ, on='date', how='left')

In [9]:
lake_winnipeg = lake_winnipeg.loc[
    pd.notnull(lake_winnipeg["in_situ_lake_water_level"])
]

In [10]:
lake_winnipeg.head()

,date,lake_name,latitude,longitude,relative_orbit,lake_water_level,in_situ_lake_water_level
79726,2019-01-10,Winnipeg,53.840506,-98.627263,112,217.231253,217.155
79727,2019-01-10,Winnipeg,53.837662,-98.628729,112,216.901952,217.155
79728,2019-01-10,Winnipeg,53.834818,-98.630195,112,216.901451,217.155
79729,2019-01-10,Winnipeg,53.831974,-98.631661,112,217.006650,217.155
79730,2019-01-10,Winnipeg,53.829131,-98.633126,112,216.928148,217.155


# Find the length of the longest track

In [11]:
# Find the longest track
counts = lake_winnipeg.groupby("date").agg(
    {
        "lake_name" : "count"
    }
)
counts

,lake_name
date,
2019-01-01,369
2019-01-05,156
2019-01-08,705
2019-01-09,272
2019-01-10,282
...,...
2021-05-14,367
2021-05-18,156
2021-05-21,704


In [12]:
max(counts["lake_name"])

920

In [13]:
max_track_length = max(counts["lake_name"])

# Adjust the data so every day has max_track_length

In [14]:
df=lake_winnipeg

In [15]:
# Taken from https://stackoverflow.com/questions/68803947/how-do-i-make-each-group-within-a-dataframe-the-same-size
df = df.set_index(["date", df.groupby("date").cumcount()])
index = pd.MultiIndex.from_product(df.index.levels, names=df.index.names)
output = df.reindex(index, fill_value=np.nan).reset_index(level=1, drop=True).reset_index()

In [16]:
# Fill in the rest of the data frame with the first entry for each date for a column
def populate_data_frame_with_first_entry_on_each_date(column):
    output.loc[
        output["date"] == date,
        column
    ] = output.loc[
        output["date"] == date,
        column
    ].iloc[0]

    
number_of_dates = len(pd.unique(output["date"]))

for j, date in enumerate(pd.unique(output["date"])):
    # We padded the array with NaNs to make each date have 920 data points
    # Find out how many non-NaNs we have. This represents the last real data
    # point on each day. E.g. on 2019-01-01 there are 369 real data points
    last_non_null_index = output.loc[
        output["date"] == date,
        "lake_water_level"
    ].count()

    # Get the lake water level on each day as a numpy array
    lake_water_level = np.array(
        output.loc[
            output["date"] == date,
            "lake_water_level"
        ]
    )

    # Get the mean and standard deviation of the non-NaN data on each day
    mean_lake_water_level = np.mean(lake_water_level[0:last_non_null_index])
    std_lake_water_level = np.std(lake_water_level[0:last_non_null_index])

    # We are going to populate the NaNs with a randomly sampled array
    # with the right standard deviation and mean
    filling_array = np.random.normal(
        loc=mean_lake_water_level,
        scale=std_lake_water_level,
        size=max_track_length - last_non_null_index # e.g. 920 - 369 = 551
    )

    # Fill the NaNs
    lake_water_level[last_non_null_index:max_track_length] = filling_array

    # Put back into the data frame
    output.loc[
        output["date"] == date,
        "lake_water_level"
    ] = lake_water_level

    populate_data_frame_with_first_entry_on_each_date("relative_orbit")
    populate_data_frame_with_first_entry_on_each_date("in_situ_lake_water_level")
    populate_data_frame_with_first_entry_on_each_date("lake_name")
    
    if j % 10 == 0 or j == number_of_dates - 1:
        percentage_complete = j/(number_of_dates - 1) * 100.
        print("%0.02f%% complete"%(percentage_complete))


0.00% complete
4.02% complete
8.03% complete
12.05% complete
16.06% complete
20.08% complete
24.10% complete
28.11% complete
32.13% complete
36.14% complete
40.16% complete
44.18% complete
48.19% complete
52.21% complete
56.22% complete
60.24% complete
64.26% complete
68.27% complete
72.29% complete
76.31% complete
80.32% complete
84.34% complete
88.35% complete
92.37% complete
96.39% complete
100.00% complete


In [17]:
# Write the data frame to csv
output.to_csv("./processed/imputed_sentinel_a_b_data.csv", index=False)

# Prepare test/train data

In [18]:
df_train = output.iloc[0:200*920]
df_test = output.iloc[201*920::]

In [19]:
def prepare_data(datain, timestep):
    # Get the lake water levels as numpy arrays
    lake_water_levels = np.array(datain["lake_water_level"])
    in_situ_lake_water_levels = np.array(datain["in_situ_lake_water_level"])
    
    # Get the number of unique dates. For e.g. in our training data, it's 200.
    number_of_dates = len(pd.unique(datain["date"]))
    
    # Number of windows we can fit into the data
    number_of_windows = number_of_dates - (2 * timestep) + 1
    
    # Sliding window across the data
    for d in range(0, number_of_dates - (2 * timestep) + 1):
        X_start = d * max_track_length # Starting index
        X_end = (d + timestep) * max_track_length # Finishing index
        
        Y = (d + np.arange(timestep)) * max_track_length # Indices for getting in-situ data
        
        if d==0:
            X_comb = lake_water_levels[X_start:X_end]
            Y_comb = in_situ_lake_water_levels[Y]
        else:
            X_comb = np.append(X_comb, lake_water_levels[X_start:X_end])
            Y_comb = np.append(Y_comb, in_situ_lake_water_levels[Y])

    # Reshape input and target arrays
    X_out = np.reshape(X_comb, (number_of_windows, timestep*max_track_length, 1))
    Y_out = np.reshape(Y_comb, (number_of_windows, timestep, 1))
    return X_out, Y_out

In [20]:
def simple_prepare_data(datain):
    # Turn our training data into a an input set of (200 x 920 x 1)
    # Turn our training data into a target set of (200 x 1 x 1)
    
    # Get the lake water levels as numpy arrays
    lake_water_levels = np.array(datain["lake_water_level"])
    in_situ_lake_water_levels = np.array(datain["in_situ_lake_water_level"])
    
    # Get the number of unique dates. For e.g. in our training data, it's 200.
    number_of_dates = len(pd.unique(datain["date"]))
    
    for d in range(number_of_dates):
        X_start = d * max_track_length
        X_end = (d + 1) * max_track_length
        Y_start = d * max_track_length
        # There is no Y-end
    
        if d == 0:
            X = lake_water_levels[X_start:X_end]
            Y = in_situ_lake_water_levels[Y_start]
        else:
            X = np.append(X, lake_water_levels[X_start:X_end])
            Y = np.append(Y, in_situ_lake_water_levels[Y_start])
    # Reshape arrays
    X_out = np.reshape(X, (number_of_dates, max_track_length, 1))
    Y_out = np.reshape(Y, (number_of_dates, 1, 1))
    return X_out, Y_out

In [21]:
# Too complicated to use sliding windows; makes it hard to interpret what happens
# X_train, Y_train = prepare_data(datain=df_train, timestep=5)
# X_test, Y_test = prepare_data(datain=df_test, timestep=5)

X_train, Y_train = simple_prepare_data(datain=df_train)
X_test, Y_test = simple_prepare_data(datain=df_test)

In [39]:
Y_train

array([[[217.144]],

       [[217.139]],

       [[217.161]],

       [[217.153]],

       [[217.155]],

       [[217.159]],

       [[217.159]],

       [[217.176]],

       [[217.186]],

       [[217.178]],

       [[217.182]],

       [[217.19 ]],

       [[217.19 ]],

       [[217.21 ]],

       [[217.22 ]],

       [[217.218]],

       [[217.229]],

       [[217.229]],

       [[217.24 ]],

       [[217.239]],

       [[217.244]],

       [[217.251]],

       [[217.246]],

       [[217.251]],

       [[217.252]],

       [[217.263]],

       [[217.273]],

       [[217.275]],

       [[217.269]],

       [[217.276]],

       [[217.279]],

       [[217.281]],

       [[217.278]],

       [[217.278]],

       [[217.28 ]],

       [[217.273]],

       [[217.279]],

       [[217.281]],

       [[217.28 ]],

       [[217.277]],

       [[217.278]],

       [[217.275]],

       [[217.272]],

       [[217.282]],

       [[217.282]],

       [[217.295]],

       [[217.304]],

       [[217.

In [23]:
X_test.shape

(49, 920, 1)

# LSTM Model

In [24]:
# Define LSTM
model = Sequential(name="LSTM-Model")
model.add(
    Input(
        shape=(X_train.shape[1], X_train.shape[2]),
        name="Input-Layer"
    )
)
model.add(
    Bidirectional(
        LSTM(
            units=32,
            activation="tanh",
            recurrent_activation="sigmoid",
            stateful=False,
        ),
        name="Hidden-LSTM-Encoder-Layer"
    )
)
model.add(
    RepeatVector(
        Y_train.shape[1],
        name="Repeat-Vector-Layer"
    )
)
model.add(
    Bidirectional(
        LSTM(
            units=32,
            activation="tanh",
            recurrent_activation="sigmoid",
            stateful=False,
            return_sequences=True
        ),
        name="Hidden-LSTM-Decoder-Layer"
    )
)
model.add(
    TimeDistributed(
        Dense(
            units=1,
            activation="linear"
        ),
        name="Output-Layer"
    )
)

2022-06-21 17:59:42.819596: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# Compile model
model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=["MeanSquaredError", "MeanAbsoluteError"],
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None
)

In [26]:
# Fit the model

# I ran the model overnight and saved the result to My_LSTM

history = model.fit(
    X_train,
    Y_train,
    batch_size=1,
    epochs=1000,
    verbose=1,
    callbacks=None,
    validation_split=0.2,
    # validation_data=(X_test, Y_test)
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=100,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=True
)

Epoch 1/1000
160/160 [==============================] - 14s 68ms/step - loss: 43330.2109 - mean_squared_error: 43330.2109 - mean_absolute_error: 208.0841
Epoch 2/1000
160/160 [==============================] - 11s 69ms/step - loss: 38122.0312 - mean_squared_error: 38122.0312 - mean_absolute_error: 195.2299
Epoch 3/1000
160/160 [==============================] - 12s 74ms/step - loss: 34804.1484 - mean_squared_error: 34804.1484 - mean_absolute_error: 186.5434
Epoch 4/1000
160/160 [==============================] - 11s 70ms/step - loss: 31856.2910 - mean_squared_error: 31856.2910 - mean_absolute_error: 178.4687
Epoch 5/1000
160/160 [==============================] - 11s 70ms/step - loss: 29150.6875 - mean_squared_error: 29150.6875 - mean_absolute_error: 170.7213
Epoch 6/1000
160/160 [==============================] - 11s 70ms/step - loss: 26641.7754 - mean_squared_error: 26641.7754 - mean_absolute_error: 163.2090
Epoch 7/1000
160/160 [==============================] - 11s 70ms/step - loss

160/160 [==============================] - 12s 73ms/step - loss: 0.0529 - mean_squared_error: 0.0529 - mean_absolute_error: 0.1853
Epoch 56/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0529 - mean_squared_error: 0.0529 - mean_absolute_error: 0.1850
Epoch 57/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0527 - mean_squared_error: 0.0527 - mean_absolute_error: 0.1844
Epoch 58/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0529 - mean_squared_error: 0.0529 - mean_absolute_error: 0.1855
Epoch 59/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0530 - mean_squared_error: 0.0530 - mean_absolute_error: 0.1851
Epoch 60/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0532 - mean_squared_error: 0.0532 - mean_absolute_error: 0.1847
Epoch 61/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0530 - mean_squared_error: 0.0530 - mean_absolute_error: 0.1859

160/160 [==============================] - 12s 73ms/step - loss: 0.0569 - mean_squared_error: 0.0569 - mean_absolute_error: 0.1944
Epoch 111/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0589 - mean_squared_error: 0.0589 - mean_absolute_error: 0.2000
Epoch 112/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0566 - mean_squared_error: 0.0566 - mean_absolute_error: 0.1909
Epoch 113/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0569 - mean_squared_error: 0.0569 - mean_absolute_error: 0.1955
Epoch 114/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0559 - mean_squared_error: 0.0559 - mean_absolute_error: 0.1929
Epoch 115/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0592 - mean_squared_error: 0.0592 - mean_absolute_error: 0.1999
Epoch 116/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0569 - mean_squared_error: 0.0569 - mean_absolute_error: 

160/160 [==============================] - 12s 73ms/step - loss: 0.0556 - mean_squared_error: 0.0556 - mean_absolute_error: 0.1913
Epoch 167/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0566 - mean_squared_error: 0.0566 - mean_absolute_error: 0.1947
Epoch 168/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0573 - mean_squared_error: 0.0573 - mean_absolute_error: 0.1940
Epoch 169/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0590 - mean_squared_error: 0.0590 - mean_absolute_error: 0.1963
Epoch 170/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0635 - mean_squared_error: 0.0635 - mean_absolute_error: 0.2099
Epoch 171/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0576 - mean_squared_error: 0.0576 - mean_absolute_error: 0.1962
Epoch 172/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0575 - mean_squared_error: 0.0575 - mean_absolute_error: 

160/160 [==============================] - 12s 73ms/step - loss: 0.0610 - mean_squared_error: 0.0610 - mean_absolute_error: 0.2000
Epoch 222/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0541 - mean_squared_error: 0.0541 - mean_absolute_error: 0.1917
Epoch 223/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0562 - mean_squared_error: 0.0562 - mean_absolute_error: 0.1895
Epoch 224/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0559 - mean_squared_error: 0.0559 - mean_absolute_error: 0.1953
Epoch 225/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0586 - mean_squared_error: 0.0586 - mean_absolute_error: 0.1977
Epoch 226/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0559 - mean_squared_error: 0.0559 - mean_absolute_error: 0.1965
Epoch 227/1000
160/160 [==============================] - 12s 73ms/step - loss: 0.0585 - mean_squared_error: 0.0585 - mean_absolute_error: 

...

160/160 [==============================] - 11s 66ms/step - loss: 0.0583 - mean_squared_error: 0.0583 - mean_absolute_error: 0.1965
Epoch 777/1000
160/160 [==============================] - 11s 66ms/step - loss: 0.0580 - mean_squared_error: 0.0580 - mean_absolute_error: 0.2005
Epoch 778/1000
160/160 [==============================] - 11s 66ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - mean_absolute_error: 0.1967
Epoch 779/1000
160/160 [==============================] - 11s 66ms/step - loss: 0.0612 - mean_squared_error: 0.0612 - mean_absolute_error: 0.2003
Epoch 780/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - mean_absolute_error: 0.1961
Epoch 781/1000
160/160 [==============================] - 11s 68ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - mean_absolute_error: 0.1937
Epoch 782/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0597 - mean_squared_error: 0.0597 - mean_absolute_error: 

160/160 [==============================] - 11s 67ms/step - loss: 0.0575 - mean_squared_error: 0.0575 - mean_absolute_error: 0.1966
Epoch 832/1000
160/160 [==============================] - 11s 68ms/step - loss: 0.0563 - mean_squared_error: 0.0563 - mean_absolute_error: 0.1895
Epoch 833/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0581 - mean_squared_error: 0.0581 - mean_absolute_error: 0.1974
Epoch 834/1000
160/160 [==============================] - 11s 68ms/step - loss: 0.0565 - mean_squared_error: 0.0565 - mean_absolute_error: 0.1955
Epoch 835/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0550 - mean_squared_error: 0.0550 - mean_absolute_error: 0.1952
Epoch 836/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0582 - mean_squared_error: 0.0582 - mean_absolute_error: 0.1992
Epoch 837/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0606 - mean_squared_error: 0.0606 - mean_absolute_error: 

160/160 [==============================] - 11s 72ms/step - loss: 0.0582 - mean_squared_error: 0.0582 - mean_absolute_error: 0.2014
Epoch 888/1000
160/160 [==============================] - 11s 69ms/step - loss: 0.0599 - mean_squared_error: 0.0599 - mean_absolute_error: 0.2009
Epoch 889/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0594 - mean_squared_error: 0.0594 - mean_absolute_error: 0.1982
Epoch 890/1000
160/160 [==============================] - 11s 68ms/step - loss: 0.0615 - mean_squared_error: 0.0615 - mean_absolute_error: 0.2062
Epoch 891/1000
160/160 [==============================] - 11s 66ms/step - loss: 0.0583 - mean_squared_error: 0.0583 - mean_absolute_error: 0.2010
Epoch 892/1000
160/160 [==============================] - 11s 66ms/step - loss: 0.0632 - mean_squared_error: 0.0632 - mean_absolute_error: 0.2049
Epoch 893/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0612 - mean_squared_error: 0.0612 - mean_absolute_error: 

160/160 [==============================] - 11s 67ms/step - loss: 0.0632 - mean_squared_error: 0.0632 - mean_absolute_error: 0.2078
Epoch 943/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - mean_absolute_error: 0.1980
Epoch 944/1000
160/160 [==============================] - 11s 66ms/step - loss: 0.0569 - mean_squared_error: 0.0569 - mean_absolute_error: 0.1955
Epoch 945/1000
160/160 [==============================] - 11s 66ms/step - loss: 0.0592 - mean_squared_error: 0.0592 - mean_absolute_error: 0.1966
Epoch 946/1000
160/160 [==============================] - 11s 68ms/step - loss: 0.0588 - mean_squared_error: 0.0588 - mean_absolute_error: 0.1964
Epoch 947/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0549 - mean_squared_error: 0.0549 - mean_absolute_error: 0.1911
Epoch 948/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0596 - mean_squared_error: 0.0596 - mean_absolute_error: 

160/160 [==============================] - 11s 67ms/step - loss: 0.0596 - mean_squared_error: 0.0596 - mean_absolute_error: 0.2010
Epoch 999/1000
160/160 [==============================] - 11s 67ms/step - loss: 0.0573 - mean_squared_error: 0.0573 - mean_absolute_error: 0.1925
Epoch 1000/1000
160/160 [==============================] - 12s 72ms/step - loss: 0.0564 - mean_squared_error: 0.0564 - mean_absolute_error: 0.1933 - val_loss: 0.1286 - val_mean_squared_error: 0.1286 - val_mean_absolute_error: 0.3453


In [27]:
model.save("My_LSTM")

INFO:tensorflow:Assets written to: My_LSTM/assets


INFO:tensorflow:Assets written to: My_LSTM/assets


In [28]:
# Predict results on training data
pred_train = model.predict(X_train)
# Predict results on test data
pred_test = model.predict(X_test)

2/2 [==============================] - 0s 26ms/step


In [38]:
model.predict(np.reshape(X_train[1], (1, 920, 1)))

1/1 [==============================] - 0s 29ms/step


array([[[217.52295]]], dtype=float32)

# Simplification: take the median on each day

In [30]:
simplified_output = output.groupby("date").agg(
    {
        "date" : "first",
        "lake_name" : "first",
        "relative_orbit" : "first",
        "lake_water_level" : "median",
        "in_situ_lake_water_level" : "first"
        # Drop lat/long since it varies for each point
    }
)